In [1]:
import cv2

## FOR MAKE THE VIDEO 

In [2]:

cap = cv2.VideoCapture(0)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter('VGame.mp4',cv2.VideoWriter_fourcc('P','I','M','1'),fps,(int(width),int(height)))

while cap.isOpened() :
    ret,frame = cap.read()

    try :
        cv2.imshow('Vgame',frame)
        videoWriter.write(frame)
    except Exception as e :
        break

    if cv2.waitKey(10) & 0xFF == ord('q') :
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows() 

# Take all the landmarks

In [3]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt
import mediapipe as mp

In [18]:
landmarks = ['class']
for val in range (1,22) :
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val)]

In [ ]:
landmarks

In [6]:
with open('coords.csv',mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [17]:
def export_landmarks(results,action) :
    try:
        keypoints = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)
        with open('coords.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e :
        pass

In [11]:
cap = cv2.VideoCapture('VGame.mp4')
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)


with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic :

    while cap.isOpened() :
        ret,frame = cap.read()

        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False 

        results = holistic.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                )
        k = cv2.waitKey(1)
        if k == 117 :
            export_landmarks(results,'V')
        if k == 100:
            export_landmarks(results,'pasV')

        cv2.imshow("Raw Webcam Feed",image)

        if cv2.waitKey(1) & 0xFF == ord('q') :
            break

cap.release()
cv2.destroyAllWindows() 

# Train the model with Scikit Learn

In [4]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('coords.csv')
X = df.drop('class',axis=1)
y = df['class']
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1234)

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [7]:
pipeline = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression()),
    'rc' : make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(),RandomForestClassifier())
}

In [14]:
fit_model = {}
for algo, pipeline in pipeline.items() :
    model = pipeline.fit(X_train,y_train)
    fit_model[algo] = model

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [15]:
for algo,model in fit_model.items() : 
    yhat = model.predict(X_test)
    # For knew the % of accuracy of our model
    print(algo,accuracy_score(y_test.values, yhat),precision_score(y_test.values,yhat,average="binary",pos_label="V"),recall_score(y_test.values,yhat,average="binary",pos_label="V"))
yhat = fit_model['rf'].predict(X_test)
with open('Vgame.pkl','wb') as f :
    pickle.dump(fit_model['rf'],f)

lr 0.8947368421052632 0.9074074074074074 0.9074074074074074
rc 0.8842105263157894 0.8771929824561403 0.9259259259259259
rf 0.9368421052631579 0.9615384615384616 0.9259259259259259
gb 0.8947368421052632 0.9230769230769231 0.8888888888888888


# Test model detection

In [9]:
with open('Vgame.pkl','rb') as f :
    model = pickle.load(f)

In [29]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = ""

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic : 
    while cap.isOpened() : 
        ret,frame = cap.read()

        image  = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = holistic.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                        )
        try :
            row = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row],columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class,body_language_prob)

            if body_language_class == 'pasV' and body_language_prob[body_language_prob.argmax()] >=0.7 :
                current_stage = 'pasV'
            elif current_stage == 'pasV' and body_language_class == 'V' and body_language_prob[body_language_prob.argmax()] >=0.7 :
                current_stage = "V"
                counter+=1

            cv2.rectangle(image,(0,0),(250,60),(0,0,255),-1)
            cv2.putText(image,'CLASS',(95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,0,0),1, cv2.LINE_AA)
            cv2.putText(image,body_language_class.split(' ')[0],(90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            cv2.putText(image,'COUNT',(180,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            cv2.putText(image,'PROB',(0,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(max(body_language_prob)),(0,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        except Exception as e :
            pass

        cv2.imshow("Vgame",image)
        
        if cv2.waitKey(1) & 0xFF == ord('q') :
            break

cap.release()
cv2.destroyAllWindows() 


V [0.7 0.3]
V [0.8 0.2]
V [0.8 0.2]
V [0.78 0.22]
V [0.79 0.21]
V [0.76 0.24]
V [0.81 0.19]
V [0.77 0.23]
V [0.79 0.21]
V [0.81 0.19]
V [0.79 0.21]
V [0.79 0.21]
V [0.79 0.21]
V [0.8 0.2]
V [0.77 0.23]
V [0.79 0.21]
V [0.83 0.17]
V [0.81 0.19]
V [0.83 0.17]
V [0.83 0.17]
V [0.83 0.17]
V [0.83 0.17]
V [0.8 0.2]
V [0.82 0.18]
V [0.79 0.21]
V [0.82 0.18]
V [0.83 0.17]
V [0.82 0.18]
V [0.81 0.19]
V [0.82 0.18]
V [0.83 0.17]
V [0.9 0.1]
